In [3]:
pip show langchain langchain-community langchain-core


Name: langchain
Version: 1.0.7
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Volumes/Development/Practise/se-practises/.venv/lib/python3.13/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
---
Name: langchain-community
Version: 0.4.1
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /Volumes/Development/Practise/se-practises/.venv/lib/python3.13/site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain-classic, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-core
Version: 1.0.5
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Volumes/Development/Practise/se-practises/.venv/lib/python3.13/site-packages
Requi

In [20]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import  HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.retrieval_qa.base import RetrievalQA
from langchain_community.chat_models import ChatOllama

In [5]:
doc_loader = PyMuPDFLoader("playwright-mcp-server.pdf")

In [6]:
doc = doc_loader.load()

In [7]:
doc_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

In [8]:
chunks = doc_splitter.split_documents(doc)
print(chunks)

[Document(metadata={'producer': 'Skia/PDF m117', 'creator': 'Chromium', 'creationdate': '2025-11-20T10:27:24+00:00', 'source': 'playwright-mcp-server.pdf', 'file_path': 'playwright-mcp-server.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-11-20T10:27:24+00:00', 'trapped': '', 'modDate': "D:20251120102724+00'00'", 'creationDate': "D:20251120102724+00'00'", 'page': 0}, page_content="QA Touch AI Test\nManagement Tool\nAccelerate your testing workflow with intelligent test case organization,\nseamless integrations, and AI-assisted insights. From planning to execution,\nQA Touch simplifies every step of your QA lifecycle.\nGet started - it's free\nBook a demo\nPlaywright MCP Server Explained:\nA Guide to Multi-Client Browser\nAutomation\nBhavani R\nMarch 29, 2025\n11 mins read\nIn this article\nWhat is Playwright?\nWhat is MCP Server?\nSetting Up Playwright MCP Server\nFeatures and Use Cases of Playwright MCP Server\n

In [9]:
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/2w/cqz3z_yn04n_r99f8b5w1frc0000gp/T/ipykernel_61390/18197047.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [10]:
texts = [c.page_content for c in chunks]
embeddings = embedder.embed_documents(texts)
embeddings_pairs = zip(texts, embeddings)

In [12]:
# vector_store = FAISS.from_embeddings(embeddings_pairs, embedding=embeddings)
vector_store = FAISS.from_texts(texts=texts, embedding=embedder)

In [21]:



retriever = vector_store.as_retriever()
llm = ChatOllama(model="llama3.1:8b")

/var/folders/2w/cqz3z_yn04n_r99f8b5w1frc0000gp/T/ipykernel_61390/944249928.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1:8b")


In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Use the context to answer.\n\nContext:\n{context}\n\nQuestion: {question}")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),  # pass the user’s string directly
    }
    | prompt
    | llm
    | StrOutputParser()
) 

In [23]:
answer = qa.invoke("what is Playwright MCP Server?")

In [24]:
answer

'According to the context, Playwright MCP (Multi-Client Protocol) Server is a server that allows multiple clients to share the same Playwright session, enabling collaborative testing, remote debugging, and enhanced performance analysis. It provides features such as multi-client interaction, distributed and parallel testing, load testing, and performance analysis. The MCP Server supports Chromium, Firefox, and WebKit browser engines and can be set up using Node.js and the Playwright package.'

In [ ]:
def self_rag(user_query):
   # first_response = llm.invoke(f"Q: {user_query}\n A: ?")
   # if 'I\'m not sure' in first_response or len(first_response) < 30:
      return retrieval_qa.run(user_query)
   # return first_response

In [ ]:

self_rag('list me the steps to implement playwright mcp')